# Michelin Rated Restaurants in France - Interactivity

In [1]:
# Module import
import pandas as pd
import geopandas as gpd

In [2]:
from Functions.functions_visualisation import dataframe_info, filter_dataframe
from Functions.plotly_visual import plot_interactive_department

----
&nbsp;
## All restaurants

In [3]:
# All restaurants
all_france = pd.read_csv("../../data/France/all_restaurants(arrondissements).csv")
dataframe_info(all_france)

Unique Regions: 13
Unique Departments: 92
Unique Arrondissements: 242
Unique Locations: 658

Shape: (1033, 15)


,name,address,location,arrondissement,department_num,department,capital,region,price,cuisine,url,award,stars,longitude,latitude
0,La Vague d'Or - Cheval Blanc St-Tropez,Plage de la Bouillabaisse,"Saint-Tropez, 83990",Draguignan,83,Var,Toulon,Provence-Alpes-Côte d'Azur,€€€€,"Creative, Modern Cuisine",https://www.chevalblanc.com/fr/maison/st-tropez/,3 MICHELIN Stars,3.0,6.626154,43.266585
1,René et Maxime Meilleur,Hameau de Saint-Marcel,"Saint-Martin-de-Belleville, 73440",Albertville,73,Savoie,Chambéry,Auvergne-Rhône-Alpes,€€€€,"Creative, Regional Cuisine",https://www.la-bouitte.com/fr/,3 MICHELIN Stars,3.0,6.513306,45.369046
2,Kei,5 rue du Coq-Héron,"Paris, 75001",1st (Louvre),75,Paris,Paris,Île-de-France,€€€€,"Modern Cuisine, Creative",https://www.restaurant-kei.fr/,3 MICHELIN Stars,3.0,2.342285,48.864395


----
&nbsp;
## GeoDataFrame (departmental level)

In [4]:
# Load GeoJSON departmental data
geo_df = gpd.read_file("../../data/France/department_restaurants.geojson")
dataframe_info(geo_df)

GeoDataFrame.
Unique Regions: 13
Unique Departments: 96

Shape: (96, 20)


,code,department,capital,region,bib_gourmand,1_star,2_star,3_star,total_stars,starred_restaurants,GDP_millions(€),GDP_per_capita(€),poverty_rate(%),average_annual_unemployment_rate(%),average_net_hourly_wage(€),municipal_population,population_density(inhabitants/sq_km),area(sq_km),locations,geometry
0,01,Ain,Bourg-en-Bresse,Auvergne-Rhône-Alpes,6,4,0,1,7,5,16726.40,25425.63,10.5,5.5,15.34,657856.0,114.2,5760.56,"{'1': [[46.3626315, 6.0258533], [46.0070045, 5...","POLYGON ((4.78021 46.17668, 4.78024 46.18905, ..."
1,02,Aisne,Laon,Hauts-de-France,0,0,0,0,0,0,12016.40,22699.26,18.0,10.5,13.92,529374.0,71.9,7362.64,"{'1': None, '2': None, '3': None}","POLYGON ((3.17296 50.01131, 3.17382 50.01186, ..."
2,03,Allier,Moulins,Auvergne-Rhône-Alpes,4,2,0,0,2,2,8278.98,24667.13,15.3,7.7,13.63,335628.0,45.7,7344.16,"{'1': [[46.1246273, 3.4192286], [46.3233297, 2...","POLYGON ((3.03207 46.79491, 3.03424 46.79080, ..."


----
&nbsp;
## Building interactive function

In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Initialize the Dash app
app = dash.Dash(__name__)

# Use geo_df to get unique regions and departments for the initial dropdowns
unique_regions = geo_df['region'].unique()
initial_departments = geo_df[geo_df['region'] == unique_regions[0]][['department', 'code']].drop_duplicates().to_dict('records')
initial_options = [{'label': f"{dept['department']} ({dept['code']})", 'value': dept['department']} for dept in initial_departments]
dept_to_code = geo_df.drop_duplicates(subset='department').set_index('department')['code'].to_dict()

app.layout = html.Div([
    dcc.Dropdown(
        id='region-dropdown',
        options=[{'label': region, 'value': region} for region in unique_regions],
        value=unique_regions[0]  # default value
    ),
    dcc.Dropdown(
        id='department-dropdown',
        options=initial_options,
        value=initial_departments[0]['department']  # default value
    ),
    dcc.Graph(id='map-display')
])

@app.callback(
    Output('department-dropdown', 'options'),
    Input('region-dropdown', 'value')
)
def update_department_dropdown(selected_region):
    departments = geo_df[geo_df['region'] == selected_region][['department', 'code']].drop_duplicates().to_dict('records')
    return [{'label': f"{dept['department']} ({dept['code']})", 'value': dept['department']} for dept in departments]

@app.callback(
    Output('map-display', 'figure'),
    Input('department-dropdown', 'value')
)
def update_map(selected_department):
    department_code = dept_to_code[selected_department]
    return plot_interactive_department(all_france, geo_df, department_code)


if __name__ == '__main__':
    app.run_server(debug=True)
